In [1]:
import numpy as np

from dask.distributed import LocalCluster

# SWYFT simulators using Dask

In [2]:
from swyft.simulator import Simulator

In [3]:
# parameter set
z = np.arange(10000)

## python-based simulator

In [4]:
def model(z):
    """ 
    Model with dummy argument
    """
    return np.random.normal()

In [5]:
# setup simulator
simulator = Simulator(model)

In [6]:
# run it with the default Dask scheduler (i.e. multiprocessing) ..
x = simulator.run(input_parameters=z)

In [7]:
# .. or use the distributed Dask scheduler
cluster = LocalCluster()
simulator.set_dask_cluster(cluster)
x = simulator.run(input_parameters=z)

In [8]:
np.mean(x), np.std(x)  # 0. and 1.

(-0.0014657086664376052, 0.9983030050718695)

## command-line simulator

In [9]:
command = '/Users/fnattino/Programs/swyft/tmp/rand_gaussian/rand_gaussian.x'

In [10]:
def set_input(z):
    """
    Dummy argument, write seed for random number generator
    """
    seed = np.random.randint(low=0, high=np.iinfo(np.int32).max)
    with open('input.inp', 'w') as f:
        f.write(str(seed))
        
def get_output(stdout, stderr):
    """
    Parse output from stdout
    """
    return float(stdout.strip())

In [11]:
# setup simulator
simulator = Simulator.from_command(
    command=command, 
    set_input_method=set_input,
    get_output_method=get_output,
    tmpdir='/tmp/'
)

In [12]:
# run the simulator
x = simulator.run(z)

In [13]:
# also try with distributed Dask scheduler
cluster = LocalCluster(threads_per_worker=1)
simulator.set_dask_cluster(cluster)
x = simulator.run(input_parameters=z)

/opt/miniconda3/envs/dark_generators/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61876 instead
  warnings.warn(


In [14]:
np.mean(x), np.std(x)  # 0. and 1.

(0.0027577676746896366, 1.0040850166553628)